In [37]:
from sklearn.model_selection import train_test_split, GridSearchCV,KFold,cross_val_score,StratifiedKFold
from sklearn.svm import SVC,LinearSVC
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
from sklearn import metrics
from statsmodels.stats.multitest import multipletests
from sklearn.multiclass import OneVsRestClassifier
from statsmodels import api as sm
from statsmodels.formula.api import ols
from bioinfokit. analys import stat as bk_stat
from sklearn.metrics import roc_auc_score
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
from scipy import signal,stats
import pandas as pd
from typing import Dict
from collections import defaultdict
from ephysvibe.trials.spikes import firing_rate,sp_constants
from ephysvibe.trials import select_trials
from ephysvibe.spike_sorting import config
from ephysvibe.structures.trials_data import TrialsData
from ephysvibe.trials.spikes import firing_rate
from ephysvibe.task import task_constants,def_task
import os 
seed = 2023

In [38]:
filepath = "/envau/work/invibe/USERS/IBOS/data/Riesling/TSCM/OpenEphys/session_struct/Riesling/lip/2023-03-03_10-59-32_Riesling_lip_e1_r1.h5"
s_path = os.path.normpath(filepath).split(os.sep)
data = TrialsData.from_python_hdf5(filepath)

In [63]:
i_neuron = 0
t_after = 850
in_out='in'
trials_b1 = np.where(np.logical_and(data.trial_error == 0, data.block == 1))[0]
shifts = data.code_samples[
    trials_b1,
    np.where(data.code_numbers[trials_b1] == task_constants.EVENTS_B1["sample_on"])[
        1
    ],
]

In [64]:
roll_sp = TrialsData.indep_roll(data.sp_samples[trials_b1,i_neuron],shifts=-shifts.astype(int), axis=1)[:,:t_after]
if np.any(np.isnan(data.neuron_cond)):
    neuron_cond = np.ones(len(data.clustersgroup))
else:
    neuron_cond=data.neuron_cond
task = def_task.create_task_frame(
    condition=data.condition[trials_b1],
    test_stimuli=data.test_stimuli[trials_b1],
    samples_cond=task_constants.SAMPLES_COND,
    neuron_cond = neuron_cond,
)
task = task[(task['i_neuron']==i_neuron)&(task['in_out']==in_out)]

In [65]:
c1_trials = task[task['sample'].str.contains('c1', case=False)]['trial_idx'].values
c5_trials = task[task['sample'].str.contains('c5', case=False)]['trial_idx'].values
o1_trials = task[task['sample'].str.contains('o1', case=False)]['trial_idx'].values
o5_trials = task[task['sample'].str.contains('o5', case=False)]['trial_idx'].values
c0_trials = task[task['sample'].str.contains('c0', case=False)]['trial_idx'].values

In [66]:
def moving_average(data:np.ndarray,win:int, step:int=1)-> np.ndarray:
    d_shape=data.shape
    d_avg = np.zeros((d_shape[0],int(np.floor(d_shape[1]/step))))
    count = 0
    for i_step in np.arange(0,d_shape[1]-step,step):
        d_avg[:,count] = np.mean(data[:,i_step:i_step+win],axis=1)
        count +=1
    return d_avg

In [67]:
win = 100
step = 1
avg_sp = moving_average(data=roll_sp,win=win, step=step)

In [68]:
c1_sp = avg_sp[c1_trials] 
c5_sp = avg_sp[c5_trials] 
o1_sp = avg_sp[o1_trials] 
o5_sp = avg_sp[o5_trials] 
c0_sp = avg_sp[c0_trials] 

In [91]:
def compute_roc_auc(group1,group2):
    #roc_score:Dict[str,list] = defaultdict(list)
    rng = np.random.default_rng(seed=seed)
    #it_seed = rng.integers(low=1, high=2023, size=a.shape[1], dtype=int)
    roc_score = []
    for n_win in np.arange(c1_sp.shape[1]):
        g2 = group1[:,n_win]
        g1 = group2[:,n_win]
        min_fr, max_fr = np.concatenate([g1,g2]).min(),np.concatenate([g1,g2]).max()
        thresholds = np.arange(min_fr,max_fr+0.01,0.01)
        y_g1, y_g2 = np.ones(len(g1)),np.zeros(len(g2))
        score=0
        fpr,tpr=[],[]
        for threshold in thresholds:
            g1_y_pred,g2_y_pred = np.zeros(len(g1)),np.zeros(len(g2))
            g1_mask,g2_mask = g1>=threshold,g2>=threshold
            g1_y_pred[g1_mask],g2_y_pred[g2_mask] = 1,1
            tp = sum(np.logical_and(y_g1==1,g1_y_pred==1))
            fn = sum(np.logical_and(y_g1==1,g1_y_pred==0))
            tpr.append(tp/ (tp+fn) )
            fp = sum(np.logical_and(y_g2==0,g2_y_pred==1))
            tn = sum(np.logical_and(y_g2==0,g2_y_pred==0))
            fpr.append(fp/ (fp+tn) )
        if len(fpr) > 1:
            fpr,tpr=np.array(fpr),np.array(tpr)
            idx_sort=fpr.argsort()
            score = metrics.auc(fpr[idx_sort],tpr[idx_sort])
            roc_score.append(score)
        else:
            roc_score.append(score)
    roc_score=np.array(roc_score)
    # fig,ax=plt.subplots(figsize=(10,3))
    # ax.plot(np.arange(len(roc_score)),roc_score['score'].values.reshape(-1))   
    # ax.set(xlabel="Time (s)", ylabel="ROC AUC score")
    # fig.tight_layout(pad=0.2, h_pad=0.2, w_pad=0.2)
        
    return roc_score


In [92]:
roc_score

array([0.55372039, 0.55505506, 0.55638972, 0.55839173, 0.55205205,
       0.5482983 , 0.55105105, 0.54571238, 0.54187521, 0.53378378,
       0.53870537, 0.54637971, 0.5482983 , 0.53828829, 0.52018685,
       0.52744411, 0.52152152, 0.51835169, 0.51651652, 0.53178178,
       0.52460794, 0.52402402, 0.51276276, 0.51651652, 0.51434768,
       0.49974975, 0.49424424, 0.48264932, 0.49332666, 0.49616283,
       0.49132466, 0.49924925, 0.49232566, 0.48740407, 0.46554888,
       0.46404738, 0.48198198, 0.45712379, 0.44994995, 0.44944945,
       0.44844845, 0.44911578, 0.45645646, 0.43309977, 0.43668669,
       0.42742743, 0.43118118, 0.43535202, 0.43043043, 0.42050384,
       0.4128295 , 0.42258926, 0.40924258, 0.39139139, 0.38813814,
       0.38972306, 0.39330998, 0.36486486, 0.35994328, 0.35785786,
       0.34976643, 0.34117451, 0.31915249, 0.31998665, 0.31431431,
       0.29621288, 0.29254254, 0.3008008 , 0.28895562, 0.27635969,
       0.27360694, 0.27377377, 0.26017684, 0.26659993, 0.24632